# Quantum Hybrid Prototype (VQC)

This notebook implements a Variational Quantum Classifier using PennyLane.

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import pandas as pd
import json
import os
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Config
N_QUBITS = 4
N_LAYERS = 2
DATA_DIR = "../data"
FEATURES_PATH = os.path.join(DATA_DIR, "features.csv")
EMBEDDINGS_PATH = os.path.join(DATA_DIR, "embeddings.json")
EMBEDDINGS_INDEX_PATH = os.path.join(DATA_DIR, "embeddings_index.csv")
WEIGHTS_PATH = "../models/vqc_weights.npy"

print("Loading data...")

In [ ]:
# Load Data
features_df = pd.read_csv(FEATURES_PATH)
with open(EMBEDDINGS_PATH, 'r') as f:
    embeddings_list = json.load(f)
embeddings = np.array(embeddings_list)
index_df = pd.read_csv(EMBEDDINGS_INDEX_PATH)
merged = pd.merge(features_df, index_df, on='id')

# Prepare X and y
indices = merged['index'].values
X_raw = embeddings[indices]

# Reduce dimensions for Quantum Circuit (N_QUBITS)
pca = PCA(n_components=N_QUBITS)
X_pca = pca.fit_transform(X_raw)

# Normalize to [-pi, pi] for rotation encoding
X_norm = (X_pca - X_pca.min(axis=0)) / (X_pca.max(axis=0) - X_pca.min(axis=0))
X_norm = X_norm * 2 * np.pi - np.pi

# Target: Binary classification for simplicity (Low vs High complexity)
# Or just Low (0) vs Not Low (1)
y_raw = merged['cyclomatic_complexity'].values
y = np.array([1 if val > 5 else -1 for val in y_raw]) # -1, 1 for quantum

X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2, random_state=42)
print(f"Data prepared. X shape: {X_train.shape}")

In [ ]:
# Quantum Circuit
dev = qml.device("default.qubit", wires=N_QUBITS)

@qml.qnode(dev)
def circuit(params, x):
    # Encoding
    for i in range(N_QUBITS):
        qml.RX(x[i], wires=i)
    
    # Variational Layers
    for l in range(N_LAYERS):
        for i in range(N_QUBITS):
            qml.Rot(params[l, i, 0], params[l, i, 1], params[l, i, 2], wires=i)
        
        # Entanglement
        for i in range(N_QUBITS - 1):
            qml.CNOT(wires=[i, i + 1])
        if N_QUBITS > 1:
            qml.CNOT(wires=[N_QUBITS - 1, 0])

    return qml.expval(qml.PauliZ(0))

def variational_classifier(params, bias, x):
    return circuit(params, x) + bias

def cost(params, bias, X, y):
    predictions = [variational_classifier(params, bias, x) for x in X]
    return np.mean((y - predictions) ** 2)

def accuracy(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)
    return loss

# Initialize weights
np.random.seed(0)
params = np.random.randn(N_LAYERS, N_QUBITS, 3, requires_grad=True)
bias = np.array(0.0, requires_grad=True)

opt = qml.NesterovMomentumOptimizer(0.01)
batch_size = 5

print("Training VQC...")
for it in range(20):
    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, len(X_train), (batch_size,))
    X_batch = X_train[batch_index]
    y_batch = y_train[batch_index]
    params, bias = opt.step(cost, params, bias, X=X_batch, y=y_batch)

    # Compute accuracy
    predictions = [np.sign(variational_classifier(params, bias, x)) for x in X_train]
    acc = accuracy(y_train, predictions)

    print(f"Iter: {it + 1:5d} | Cost: {cost(params, bias, X_batch, y_batch):0.7f} | Accuracy: {acc:0.7f}")

# Save weights
np.save(WEIGHTS_PATH, {'params': params, 'bias': bias})
print(f"Weights saved to {WEIGHTS_PATH}")